<a href="https://colab.research.google.com/github/bkokalari/Doctoranytime-Automatic-Review-Acceptance-Model/blob/main/Greek/ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install greek-stemmer
!python -m spacy download el # greek lang
!python -m spacy download el_core_news_sm

  Created wheel for greek-stemmer: filename=greek_stemmer-0.1.1-cp36-none-any.whl size=6738 sha256=e4e950127a68c0e9f78c288dfb90e6dba9c99d9b80d66bccc46aff194dbf56ae
  Stored in directory: /root/.cache/pip/wheels/b9/e5/39/735c9926a0da4bc26812631945f8394a697baf31d0423d154a
Successfully built greek-stemmer
     |████████████████████████████████| 11.4MB 1.7MB/s 
  Created wheel for el-core-news-sm: filename=el_core_news_sm-2.2.5-cp36-none-any.whl size=11422788 sha256=aa4dfa20b085cec73e7554380049dedec4a87423f006edca78af07c22d26239f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5b16i_2i/wheels/70/a1/c5/6690d6b524d87e287a8070cf957f834fb1b1665b9ede11348b
Successfully built el-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('el_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/el_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/el
You can now load the model via spacy.load('el')
✔ Download and installatio

In [ ]:
import nltk

nltk.download('words')

from nltk.corpus import words
english = words.words()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
from IPython.core.debugger import set_trace
from spacy.lang.el.stop_words import STOP_WORDS
from numpy import nan
import spacy
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from greek_stemmer import GreekStemmer
from spacy.lang.el import Greek
import tensorflow as tf
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
revs = pd.read_csv("/content/drive/My Drive/Colab Notebooks/new_dataset/augmented_greek.csv") 
revs.sample(5)
revs.dropna()
#train = revs[1:100000]
revs['review'] = revs['review'].astype(str)

In [ ]:
!python3 -m spacy download el_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('el_core_news_sm')


### Tokenize and build vocab

In [ ]:
import el_core_news_sm

# Load the small english model. 
# Disable the advanced NLP features in the pipeline for efficiency.
nlp = el_core_news_sm.load(disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

#@Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#@Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    if (not token.is_punct):
#    if (token.is_alpha or token.is_digit):
      normalized = token.text.lower().strip()
      normalized_tokens.append(normalized)
  return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))  

In [ ]:
# Use the tokenizer to extract all tokens from the body of the posts.
# Flatten the tokens in the post into a single list of all the tokens.
import itertools
all_tokens = []
all_posts_tokenized = revs.review.apply(tokenize_normalize)
all_tokens = list(itertools.chain.from_iterable(all_posts_tokenized))
print("Num tokens: ", len(all_tokens))

Num tokens:  1944749


In [ ]:
import collections

class SimpleDictionary(object):

  # Add special "padding" tokens as well as unk.
  START_TOKEN = "<p>"
  END_TOKEN = "</p>"
  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    self.token_counts = collections.Counter(tokens)
    self.N = sum(iter(self.token_counts.values()))

    # Leave space for "<p>", "</p>", and "<unk>"
    top_counts = self.token_counts.most_common(None if size is None else (size - 3))
    vocab = ([self.START_TOKEN, self.END_TOKEN, self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in iter(self.id_to_word.items())}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience create the vocab set
    self.vocab_set = set(iter(self.word_to_id.keys()))

    # Store special IDs
    self.START_ID = self.word_to_id[self.START_TOKEN]
    self.END_ID = self.word_to_id[self.END_TOKEN]
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]
  
  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def sentence_to_ids(self, words):
    return [self.START_ID] + self.words_to_ids(words) + [self.END_ID]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))

In [ ]:
dictionary = SimpleDictionary(all_tokens)
print("Vocabulary size: %d unique words" % dictionary.size)

Vocabulary size: 61301 unique words


### Unigram

In [ ]:
#@title
from collections import defaultdict
import numpy as np

class UnigramLM(object):
  
  def __init__(self, dictionary):
    # Compute the probabilities and store them in a data structure.
    # prob = {}
    self.vocab_set = dictionary.vocab_set.copy()
    self.diction = defaultdict(lambda: 0.0)

    for x in dictionary.token_counts:

        self.diction[x] = dictionary.token_counts[x] / dictionary.N

    # self.diction = prob


    #self.vocab_set = prob

        
  # Compute the conditional probability of the next token for the unigram model. 
  def next_token_conditional_prob(self, previous_words, next_word):

    if next_word in self.vocab_set:
      return self.diction[next_word]
      #return self.vocab_set[next_word] 
    else:
      return 0
    
  # Compute the probability of a sequence, P(w1, ..., wn)
  # When Verbose is true, it prints the probability for each
  # token in the sequence.

  def sequence_probability(self, sequence, verbose=False):

    seq_prob = 1

    for word in sequence:

      p = self.diction[word] 
      if (verbose == True):
        print(word,p)
      seq_prob *= p
        #print(seq_prob)
    return seq_prob


In [ ]:
unigram_lm = UnigramLM(dictionary)

In [ ]:
# Try computing the unigram probabilities for indivudal words.
print("Pr(ο):", unigram_lm.next_token_conditional_prob(None, 'ο'))
print("Pr(καλός):", unigram_lm.next_token_conditional_prob(None,'καλός'))
print("Pr(γιατρός):", unigram_lm.next_token_conditional_prob(None,'γιατρός'))

# And for the same word, but with different contexts
print(unigram_lm.next_token_conditional_prob(["το", "η", "αυτό", "γιατρός", "καλός"], "αλλά"))
print(unigram_lm.next_token_conditional_prob(["το"], "αλλά"))


Pr(ο): 0.00693354258055924
Pr(καλός): 0.00545983054882661
Pr(γιατρός): 0.013554962619854798
0.0020136274655495386
0.0020136274655495386


### Trigram

In [ ]:
#@title
from collections import defaultdict
import numpy as np

class SimpleTrigramLM(object):
    """Simple Trigram LM"""
    order_n = 3

    def __init__(self, tokens):
        """Build our trigram model.
        Args:
          tokens: (list or np.array) of training tokens
        Returns:
          None
        """
        print("Num tokens: ", len(tokens))
        
        # Raw trigram counts over the corpus.
        # c(w | w_1 w_2) = self.counts[(w_2,w_1)][w]
        # Be sure to use tuples (w_2,w_1) as keys, *not* lists [w_2,w_1]
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))

        # Map of (w_1, w_2) -> int
        # Entries are c( w_2, w_1 ) = sum_w c(w_2, w_1, w)
        self.context_totals = dict()

        # Track unique words seen, for normalization
        self.vocab = set()

        # Iterate through the word stream once
        # Compute trigram counts 
        # This is a sliding window over each word.
        w_1, w_2 = None, None
        for word in tokens:
            self.vocab.add(word)
            if w_1 is not None and w_2 is not None:
                self.counts[(w_2,w_1)][word] += 1
            # Update context
            w_2 = w_1
            w_1 = word

        for context, ctr in iter(self.counts.items()):  
            self.context_totals[context] = sum(iter(ctr.values())) 

        # Total vocabulary size, for normalization
        self.V = len(self.vocab)

    def next_token_conditional_prob(self, word, seq):
        """Next token conditional probability.
        Args:
          word: (string) w in P(w | w_1 w_2 )
          seq: (list of string) [w_1, w_2, w_3, ...]
        Returns:
          (float) P_k(w | w_1 w_2), according to the model
        """
        context = tuple(seq[-2:])  # (w_2, w_1)
        cw = self.counts.get(context, {}).get(word, 0)  
        numerator = cw 
        cc = self.context_totals.get(context, 0)  
        denominator = cc  
        return numerator / denominator  
      
    def sequence_probability(self, seq, verbose=False):
      """Compute log probability (base 2) of the given sequence."""
      context_size = self.order_n - 1
      score = 0.0
      count = 0
      # Start at third word, since we need a full context.
      for i in range(context_size, len(seq)):
        context = seq[i-context_size:i]
        context_prob = self.next_token_conditional_prob(seq[i], context)
        s = np.log2(context_prob)
        # DEBUG.
        if verbose:
            print ("log P(%s | %s) = %.03f" % (seq[i], " ".join(context), s))
        score += s
        count += 1
       
      return score, count      

In [ ]:
import time
t0 = time.time()
print ("Building trigram LM..."),
lm = SimpleTrigramLM(all_tokens)
print ("done in %.02f s" % (time.time() - t0))

Building trigram LM...
Num tokens:  1944749
done in 5.75 s


In [ ]:
print ("Most frequent tri-grams:")
flat_counts = defaultdict(lambda: 0.0)
# This code puts the contexts back together with their word counts
for (context, counts) in lm.counts.items():
  for (word, count) in counts.items():
    flat_counts[context, word] = count

sorted_trigrams = sorted(flat_counts.items(), key=lambda k_v: k_v[1], reverse=True)

for (word, count) in sorted_trigrams[:50]:
    print("\"%s\": %d" % (word, count))

Most frequent tri-grams:
"(('πολύ', 'καλός'), 'γιατρός')": 3446
"(('τον', 'συστήνω'), 'ανεπιφύλακτα')": 3206
"(('μου', 'ενέπνευσε'), 'εμπιστοσύνη')": 2909
"(('την', 'περίπτωση'), 'μου')": 2011
"(('το', 'πρόβλημα'), 'μου')": 1871
"(('την', 'περίπτωσή'), 'μου')": 1857
"(('με', 'έκανε'), 'να')": 1776
"(('τον', 'συνιστώ'), 'ανεπιφύλακτα')": 1486
"(('πολύ', 'καλή'), 'γιατρός')": 1390
"(('έμεινα', 'πολύ'), 'ευχαριστημένη')": 1360
"(('με', 'την'), 'περίπτωση')": 1341
"(('στην', 'ώρα'), 'του')": 1334
"(('σου', 'εμπνέει'), 'εμπιστοσύνη')": 1308
"(('από', 'την'), 'πρώτη')": 1302
"(('σε', 'κάνει'), 'να')": 1236
"(('ο', 'γιατρός'), 'ήταν')": 1216
"(('το', 'πρόβλημά'), 'μου')": 1191
"(('με', 'το'), 'πρόβλημα')": 1176
"(('την', 'συστήνω'), 'ανεπιφύλακτα')": 1054
"(('θα', 'τον'), 'πρότεινα')": 1047
"(('τον', 'προτείνω'), 'ανεπιφύλακτα')": 1031
"(('μου', 'και'), 'μου')": 1018
"(('έκανε', 'να'), 'νιώσω')": 1009
"(('με', 'την'), 'περίπτωσή')": 989
"(('την', 'πρώτη'), 'στιγμή')": 986
"(('πολύ', 'καλός'),

In [ ]:
sorted_trigrams

[((('πολύ', 'καλός'), 'γιατρός'), 3446.0),
 ((('τον', 'συστήνω'), 'ανεπιφύλακτα'), 3206.0),
 ((('μου', 'ενέπνευσε'), 'εμπιστοσύνη'), 2909.0),
 ((('την', 'περίπτωση'), 'μου'), 2011.0),
 ((('το', 'πρόβλημα'), 'μου'), 1871.0),
 ((('την', 'περίπτωσή'), 'μου'), 1857.0),
 ((('με', 'έκανε'), 'να'), 1776.0),
 ((('τον', 'συνιστώ'), 'ανεπιφύλακτα'), 1486.0),
 ((('πολύ', 'καλή'), 'γιατρός'), 1390.0),
 ((('έμεινα', 'πολύ'), 'ευχαριστημένη'), 1360.0),
 ((('με', 'την'), 'περίπτωση'), 1341.0),
 ((('στην', 'ώρα'), 'του'), 1334.0),
 ((('σου', 'εμπνέει'), 'εμπιστοσύνη'), 1308.0),
 ((('από', 'την'), 'πρώτη'), 1302.0),
 ((('σε', 'κάνει'), 'να'), 1236.0),
 ((('ο', 'γιατρός'), 'ήταν'), 1216.0),
 ((('το', 'πρόβλημά'), 'μου'), 1191.0),
 ((('με', 'το'), 'πρόβλημα'), 1176.0),
 ((('την', 'συστήνω'), 'ανεπιφύλακτα'), 1054.0),
 ((('θα', 'τον'), 'πρότεινα'), 1047.0),
 ((('τον', 'προτείνω'), 'ανεπιφύλακτα'), 1031.0),
 ((('μου', 'και'), 'μου'), 1018.0),
 ((('έκανε', 'να'), 'νιώσω'), 1009.0),
 ((('με', 'την'), 'περίπτ

### Word Corrector

In [ ]:
class SimpleSpellingCorrector(object): 
  
  def __init__(self, lm):
    self.lm = lm
  
  def P(self, word): 
    "Unigram probability of `word`."
    return self.lm.diction[word]
    
  def correction(self, word): 
    "Most probable spelling correction for word."
    return max(self.candidates(word), key=self.P)

  def candidates(self, word): 
    "Generate the union of the sets of spelling correction candidates for a word."
    return self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(words)) or [word]

  def known(self, words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in self.lm.vocab_set)

  def edits1(self, word):
    "All edits that are one edit away from `word`."
    letters    = 'αβγδεζηθικλμνξοπρστυφχψωabcdefghijklmnopqrstuvxwz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

  def edits2(self, word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

In [ ]:
word = "ιαρτος"

corrector = SimpleSpellingCorrector(unigram_lm)
candidates = corrector.candidates(word)
print("Spelling candidates for:", word)
for candidate in candidates:
  print("{} \t{:.9f}".format(candidate, corrector.P(candidate)))

# The correct takes the word that has the highest probability (occurs most often).
correction = corrector.correction(word)
print("\nSelected correction:", correction, corrector.P(correction))

Spelling candidates for: ιαρτος
γιαρτος 	0.000000514
ιατρος 	0.000426276

Selected correction: ιατρος 0.00042627609012782627


In [ ]:
# Simple function that takes a string as input
# and runs the spelling corrector for each token. 
# Returns a list of spell-corrected tokens.
def spell_correct(string, corrector):
  corrections = list()
  tokens = tokenize_normalize(string)
  for t in tokens: 
    correction = corrector.correction(t)
    corrections.append(correction)
  return corrections 

string = "ο ιαρτος διν ηταα κλος"
print(spell_correct(string, corrector))

['ο', 'ιατρος', 'δεν', 'ηταν', 'καλος']


### Text generator

In [ ]:
def sample_next(lm, seq):
    """Sample a word from the conditional distribution."""
    # This looks through each possible next word in the vocab and computes its 
    # conditional probability with the current sequence.
    probs = [lm.next_token_conditional_prob(token, seq) for token in lm.vocab]
    
    # Pick a word at random according to its conditional probability
    return np.random.choice(list(lm.vocab), p=probs)

In [ ]:
# Given it the start sequence to indicate the start of a post. 
   
def hallucinate_text(start, max_length):
  sequence = list()
  sequence.extend(start)
  for i in range(max_length - len(start)):
      sequence.append(sample_next(lm, sequence))
  #print (" ".join(sequence))
  #print ("[{1:d} tokens; log P(seq): {0:.02f}]".format(lm.sequence_probability(seq)))
  return sequence
    
# Maximum length of sequence to generate. 
max_length = 5

# Number of sequences to generate
num_sequences = 5


start = ["δεν", "ηταν"] # Needs to be an n-gram that occurs in our collection
for _ in range(num_sequences):
  sequence = hallucinate_text(start, max_length)
  print (" ".join(sequence))
    

δεν ηταν ενημερωμενη για το
δεν ηταν σημαντικο που ηρεμισε
δεν ηταν ενημερωμενη οτι ειμαι
δεν ηταν ενδεικτική αλλά ακριβής
δεν ηταν στο περίπου αλλά


In [ ]:
start = ["δεν", "μου"] # Needs to be an n-gram that occurs in our collection
for _ in range(num_sequences):
  sequence = hallucinate_text(start, max_length)
  print (" ".join(sequence))
    

δεν μου είναι ευγενικός και
δεν μου το έβγαλε πολύ
δεν μου είπε απλώς να
δεν μου έδωσε δυνατότητα επιλογής
δεν μου ενέπνευσε εμπιστοσύνη με
